<a href="https://colab.research.google.com/github/yuvi-s64/YuviN-DataScience-GenAI-Submissions/blob/main/4_01_Linear_Regression_(statistics_vs_machine_learning)_COMPLETED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 4.01 Linear Regression (statistics vs. machine learning)
In this notebook, as we did in the lecture, we will be looking at the simple linear regression algorithm, but from two contrasting methodological approaches:
1. a traditional stastical / scientific approach;
2. a machine learning (ML) approach.

Through this comparison we can hopefully see a bit more clearly how the machine learning approach works, and how its slightly differring goals lead to slightly differring steps.

We'll begin by importing the libraries/packages and the data:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# package / library for the traditional implementation
import statsmodels.api as sm

# package / library for the ML implementation
from sklearn.linear_model import Lasso

# error metrics to assess the model
from sklearn.metrics import r2_score

df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv", header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


As this tutorial focuses on the differences in model building, we will forgo the usual cleaning/EDA/feature engineering steps and use the data as-is.

In our data set we have 13x $x$ values (independent variables/features) as columns 0-12, and one $Y$ value (dependent variable/target) as column 13. We'll seperate that out ready for modelling:

In [2]:
x_values = df.drop([13], axis = 1)
print(f'X values: \n {x_values.head()}\n')
print('\n')

y_value = df[13]
print(f'Y value: \n {y_value[0:5]}')

X values: 
         0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12  
0  396.90  4.98  
1  396.90  9.14  
2  392.83  4.03  
3  394.63  2.94  
4  396.90  5.33  



Y value: 
 0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: 13, dtype: float64


## Linear Regression in Statistics
As we can see, our $Y$ value is a float (real number) and a sensible candidate for linear regression (assuming our relationship is indeed linear). More specifically, we are going to be a build a model of the form:

$ Y = α + \beta_1x_1 + \beta_2x_2 + [...] + \beta_{13}x_{13}$

Where:
<br>
* $Y$ is our dependent variable (target)
* $\alpha$ is the intercept and
* the various $\beta$ values (1 to 13) represent the coefficient of the corresponding $x$ values (1 to 13).
<br>

#### Solving the Model
As per the slides, linear regression will learn this model by solving the below equation:
<br><br>
$OLS = minimise\;\Sigma {(y_i - \hat{y_i})^2}$
<br><br>
We will look at this step by step beginning with the $\Sigma {(y_i - \hat{y_i})^2}$ part. This means, for every value of $y$ we subtract the value predicted by our model $\hat{y}$ from the real value ($y$) to give a measure of the raw gap between real value and the regression line of our model. We do this for each real value of $y$ and then sum / add them all together to get a total measure of error across the dataset (denoted by the $\Sigma$ symbol). However, the values calculated by doing this subtraction can be negative or positive. E.g. assume our data shows a value of $y$=10 when $x$=1 and $y$=20 when $x$=2. Our prediction ($\hat{y}$), however, is that $y$=12 when $x$=1 and $y$=18 when $x$=2. This means we would calculate:
* For $x$=1 then $y - \hat{y} = 10 - 12 = -2$
* For $x$=2 then $y - \hat{y} = 20 - 18 = 2$

Taking the sum of these ($\Sigma$) would give us -2 + 2 ... which is zero. I.e. no error which is obviously incorrect. Instead, we will square the result of our subtraction changin the calculation to:
* For $x$=1 then $(y - \hat{y})^2 = (10 - 12)^2 = -2^2 = 4$
* For $x$=2 then $(y - \hat{y})^2 = (20 - 18)^2 = 2^2 = 4$

I.e. by squaring the result of these calculations we guarantee that all our outputs will be positive. Now the sum of these ($\Sigma$) would give us 4 + 4 = 8 ... a much better representation of the amount of error than our previous zero!

However, this there are many possible lines we could draw to calculate $\hat{Y}$, how do we know which is the right one? The $minimise$ part of the above equation does this for us, by using an optimisation method (you can learn more about it [here](https://en.wikipedia.org/wiki/Ordinary_least_squares#Estimation) if interested but we'll consider that out of scope for this tutorial). This means which ever line has the least error, as measured by our $\Sigma {(y_i - \hat{y_i})^2}$ metric, will be automatically selected.   

With this background behind us, we'll next build this model (using _statsmodel_ for the traditional / statistical approach) by fitting the algorithm to the data:

In [3]:
mod = sm.OLS(y_value, x_values)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                     13   R-squared (uncentered):                   0.959
Model:                            OLS   Adj. R-squared (uncentered):              0.958
Method:                 Least Squares   F-statistic:                              891.3
Date:                Sun, 07 Dec 2025   Prob (F-statistic):                        0.00
Time:                        14:52:50   Log-Likelihood:                         -1523.8
No. Observations:                 506   AIC:                                      3074.
Df Residuals:                     493   BIC:                                      3128.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Following our statistical appraoch there are several necessary steps in interpreting these results:


1.  _Examining the 'goodness of fit' metrics_: Depending on our prefered method, this could be examing $R^2$, adjusted $R^2$, AIC, BIC or some combination of these. In this case we will limit ourselves to just the first ($R^2$). Looking at the table in the ouputs, at 96% this
is clearly high, and strong support for our implicit theory that this model would represent an appropriate data generation process for our observations;
2.   _Evaluate the hypotheses tests_: Our overall $F$-statistic is ~0 which is obviously passing our hypothesis test at 95% confidence. Most of our independent variables report $p$-values below 0.05 with the exception of 3, 5 and 7. We would probably want to experimentally remove these to arrive at a model where all $x$ values are significant;
2.   _Evaluate the other metrics and information_: In a 'proper' analysis we would also want to consider measures such as the skewdness and kurtosis of the data, and we also get a helpful warning that there may be some multicolinearity in the data. Inspection of the correlation matrix reveals this is the case, so in the real-world we would have further work required to finsalise this model.

However, for the purposes of this tutorial we have reached our goal! We have fitted a linear regression model (in the statistical style) and performed an initial interpretation of the results. Our step is do this all again, but this time with a machine learning mindset.



## Linear Regresssion in Machine Learning
In our ML implementation, much of the pre-work (data cleaning, EDA) would be the same, but again skipped for simplicity. However, unlike in a statistical approach, our focus moves more the measuring the performance of the model on unseen data, i.e. our ability to learn something from one dataset and apply that to another, meaning we need an additional data preparation step.

###Train-Test Split
As per the slides, we will split the data such that part can be used for training the model (learning the line), and part can be reserved for testing the model. We can do this with just a couple of lines of code:

In [4]:
# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_values, y_value, test_size = 0.2, random_state=1234)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In our code we have split using an 80:20 ratio (80% of the data for training; 20% for testing). We specify _random\_state_ as a fixed number so that this split will be the same each time (the splitting algorithm is based on random numbers). We can confirm this has worked by looking at the size of our different datasets:


*   `X_train` (the $x$ values we use for training) is 404 rows and 13 columns. 13x columns is what we would expect;
*   `X_test` (the $x$ values we use for testing) is 102 rows and 13 columns. Again we expect the 13x columns, but also we can compare the number of rows with _X\_train_ ... 102 rows is approximately 20% of the total;
*  `Y_train` (the $Y$ values we use for training) is 404 rows and a single column - again, what we would expect;
*   `Y_test` (the $Y$ values we use for testing) is 102 rows and a single columns. All seems to be correct!
<br>
<br>

###Regularisation
Now we can turn our attention to the model. Our linear regression will remain the same, but we will use a modified algorithm to determine it. Specifically we will use $L1$ regression (LASSO) where the objective function (cost/loss function) is:

$OLS\ objective + \alpha \cdot  \Sigma |\beta_i|$

We add a penalty to the ordinary least squares (OLS) objective (described above) to limit the sum of the absolute values of the co-efficients ($\beta$ weights). Absolute values, denoted by the "|" symbols in $|\beta|$, means that we change any negatives values to their positive equivalent - e.g. -2 becomes 2; -1 becomes 1; while 4 stays as 4.

Because OLS is a _minimisation_ objective (we want the line with the smallest error), this means we will be minimising the sum ($\Sigma$) of the $\beta$ weights as well as OLS. I.e. we want a line that can closely fit the data whilst not having any particularly large and noisy weights. If we have a large weight for $\beta_n$, this means that any random errors in $x_n$ will be magnified, as we are multiplying two, meaning our model can have more variance.

We can see the amount of influence this penalty has is controlled by the hyperparameter $\alpha$. If we had a very large value of $\alpha$, the model will be encouraged to reduce all $\beta$ weights towards zero; if we have a very small value of $\alpha$ (approaching zero), the model will be basically the same as normal linear regression (i.e. the penalty term is ignored). Hyperparameters are something we will return to later in the module but for the moment we will just use an arbitrary value of 0.25.

With this in mind we can learn our model from the data:

In [5]:
# configure the model
l1_algo = Lasso(alpha=0.25)
l1_algo

Lasso(alpha=0.25)

In [6]:
# learn the model from the training data
l1_model = l1_algo.fit(X_train, Y_train)

# predict the data
predict = l1_model.predict(X_test)

# calculate R^2 by comparing the predicted values and real values
r2 = r2_score(Y_test, predict)
print(f'R^2 score is {round(r2, 2)}')

R^2 score is 0.76


We can see that $R^2$ is lower than our previous (statistical) model which achieved 96%. However, we need to recall that in this case we are testing the model against previously unseen data ... a much harder problem. Overall an $R^2$ at 76% is generally going to be considered quite high for this kind of task - so good news. That is not to say that this is the "optimal" model, and in practice we would experiment further (for example, trying different values of $\alpha$ to control how much our model favours OLS and the line of least error, versus how much we penalise large $\beta$ weights with the L1 penalty).

You may also notice the conspicuous absence of any hypothesis testing ($p$-values). Typically we would not report or test these in a machine learning approach. We do not have the same underlying (implicit) beliefs of the model as generator of the data, and ultimately no hypotheses to test (in the traditional sense). In essence the model is the hypothesis (our belief that this model will be able to score ($R^2$ in this case) reasonably well on unseen/test data). In machine learning terminology, you will often hear the possible models to be tested described as the 'hypothesis space'. Given this change in focus, $p$-values are simply irrelevant.

However, we may also want to inspect the $\beta$ values of the model:

In [7]:
# print the beta values of the model (co-efficients)
betas_l1 = l1_model.coef_
counter = 0
for col in x_values.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(f"{col} : {round(betas_l1[counter], 4)}")
    counter +=1

Beta weights/co-efficients - LASSO
-----------------------------------------
0 : -0.0831
1 : 0.0657
2 : -0.0228
3 : 0.0
4 : -0.0
5 : 2.3925
6 : -0.0087
7 : -1.2545
8 : 0.305
9 : -0.0165
10 : -0.7983
11 : 0.011
12 : -0.6034


We may observe two things. Firstly we can see the $\beta$ weights for feature 3 and 4 have been set at zero - effectively deleting these features entirely (any number multipled by zero is zero). From the class metaphor, the algorithm has decided these two decision makers are not improving the decision quality and their influence increases the variance of the model.

Secondly, we will notice that nearly all of the $\beta$ weights are smaller (closer to zero) than in the statistical appraoch of the last Notebook. For instance, feature 5 had a $\beta$ weight of 5.93 in the original model, compared with 2.39 here (less than half). Again, this fits with our metaphor - the algorithm has tried to ensure that no voices are too loud in  our group discussion. I.e. our model will be less likely to over-respond to excess noise in any particular feature as the associated $\beta$ weight is smaller.

So there we have it. Hopefully the comparison of the two approaches gives you some insight into how these two methodologies differ, and how the statistical approach has effectively been adapted to the supervised machine learning problem ... building models that can learn from data.